# Anthropic Structured Outputs with output_format

This notebook demonstrates how to use Anthropic's structured outputs feature using the `output_format` parameter. Anthropic's structured outputs allow you to enforce a specific JSON schema for the model's response, ensuring consistent and validated output.

````{=mdx}
:::info Requirements
Install `ag2` with Anthropic support:

pip install -U ag2[anthropic]
```

> **Note:** Structured outputs require `anthropic` SDK >= 0.39.0. If you have an older version, the client will automatically fall back to legacy JSON mode with a warning.

For more information, please refer to the [installation guide](https://docs.ag2.ai/latest/docs/user-guide/basic-concepts/installing-ag2).
:::
````

## Understanding output_format

Anthropic's `output_format` parameter accepts a JSON schema that defines the structure of the response. The schema must follow this format:

n
{
    "type": "json_schema",
    "schema": {
        "type": "object",
        "properties": {
            "field_name": {"type": "string"}
        },
        "required": ["field_name"],
        "additionalProperties": false
    }
}
```

## Example 1: Using output_format directly

Let's create a simple example that extracts contact information using structured outputs.

In [ ]:
import os

from autogen import AssistantAgent, UserProxyAgent

# Define the output format schema
output_format = {
    "type": "json_schema",
    "schema": {
        "type": "object",
        "properties": {
            "name": {"type": "string"},
            "email": {"type": "string"},
            "age": {"type": "integer"},
            "interests": {"type": "array", "items": {"type": "string"}},
        },
        "required": ["name", "email", "age", "interests"],
        "additionalProperties": False,
    },
}

# Create LLM config with output_format
llm_config = {
    "config_list": [
        {
            "api_type": "anthropic",
            "model": "claude-sonnet-4-5",
            "api_key": os.getenv("ANTHROPIC_API_KEY"),
            "output_format": output_format,  # Direct output_format usage
        }
    ]
}

# Create agents
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
)

assistant = AssistantAgent(
    name="ContactExtractor",
    llm_config=llm_config,
    system_message="Extract contact information from user messages and return it in the specified JSON format.",
)

# Test the structured output
result = user_proxy.initiate_chat(
    assistant,
    message="My name is John Doe, I'm 30 years old, my email is john@example.com, and I love Python and AI.",
    max_turns=1,
)

print(result.chat_history[-1]["content"])

## Example 2: Using response_format (converted to output_format)

You can also use `response_format` with a Pydantic model, which will be automatically converted to `output_format`:

In [ ]:
from pydantic import BaseModel


# Define a Pydantic model
class ContactInfo(BaseModel):
    name: str
    email: str
    age: int
    interests: list[str]


# Create LLM config with response_format (will be converted to output_format)
llm_config = {
    "config_list": [
        {
            "api_type": "anthropic",
            "model": "claude-sonnet-4-5",
            "api_key": os.getenv("ANTHROPIC_API_KEY"),
            "response_format": ContactInfo,  # Pydantic model - auto-converted
        }
    ]
}

assistant = AssistantAgent(
    name="ContactExtractor", llm_config=llm_config, system_message="Extract contact information from user messages."
)

result = user_proxy.initiate_chat(
    assistant,
    message="I'm Jane Smith, 28 years old, email jane@example.com, interested in machine learning and data science.",
    max_turns=1,
)

print(result.chat_history[-1]["content"])

## Example 3: Complex nested schema

Here's an example with a more complex nested structure for math problem solving:

In [ ]:
# Define a complex output format for step-by-step math reasoning
math_output_format = {
    "type": "json_schema",
    "schema": {
        "type": "object",
        "properties": {
            "steps": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "step_number": {"type": "integer"},
                        "explanation": {"type": "string"},
                        "calculation": {"type": "string"},
                    },
                    "required": ["step_number", "explanation", "calculation"],
                    "additionalProperties": False,
                },
            },
            "final_answer": {"type": "string"},
            "confidence": {"type": "string", "enum": ["high", "medium", "low"]},
        },
        "required": ["steps", "final_answer", "confidence"],
        "additionalProperties": False,
    },
}

llm_config = {
    "config_list": [
        {
            "api_type": "anthropic",
            "model": "claude-sonnet-4-5",
            "api_key": os.getenv("ANTHROPIC_API_KEY"),
            "output_format": math_output_format,
        }
    ]
}

math_assistant = AssistantAgent(
    name="MathSolver", llm_config=llm_config, system_message="Solve math problems step by step, showing your reasoning."
)

result = user_proxy.initiate_chat(math_assistant, message="Solve for x: 3x + 7 = 22", max_turns=1)

print(result.chat_history[-1]["content"])

## Example 4: Using the AnthropicClient directly

You can also use the `AnthropicClient` directly for more control:

In [ ]:
from autogen.oai.anthropic import AnthropicClient

# Initialize the client
client = AnthropicClient(api_key=os.getenv("ANTHROPIC_API_KEY"))

# Define output format
output_format = {
    "type": "json_schema",
    "schema": {
        "type": "object",
        "properties": {
            "summary": {"type": "string"},
            "key_points": {"type": "array", "items": {"type": "string"}},
            "sentiment": {"type": "string", "enum": ["positive", "neutral", "negative"]},
        },
        "required": ["summary", "key_points", "sentiment"],
        "additionalProperties": False,
    },
}

# Create a completion with structured output
response = client.create({
    "model": "claude-sonnet-4-5",
    "messages": [
        {
            "role": "user",
            "content": "Analyze this text: 'The new product launch was a huge success. Sales increased by 50% and customer feedback has been overwhelmingly positive.'",
        }
    ],
    "output_format": output_format,
    "max_tokens": 500,
})

print(response.choices[0].message.content)

## Key Points

1. **output_format vs response_format**: 
   - `output_format` is the native Anthropic parameter (dict with `type: "json_schema"`)
   - `response_format` can be a Pydantic model or dict, and will be automatically converted to `output_format`

2. **Schema Requirements**:
   - Must have `type: "object"` at the root
   - Should set `additionalProperties: false` to enforce strict schema
   - All required fields must be listed in the `required` array

3. **SDK Version**: Requires `anthropic` SDK >= 0.39.0. Older versions will fall back to legacy JSON mode with a warning.

4. **Beta API**: The client automatically uses Anthropic's beta API (`beta.messages.create`) when structured outputs are enabled, adding the required `betas: ["structured-outputs-2025-11-13"]` header.

For more information, see the [Anthropic structured outputs documentation](https://docs.anthropic.com/en/docs/build-with-claude/structured-outputs).